In [18]:
#general plan: get death, cases, and vaccine data for all counties combined by month
#and then plot the three of them against each other in a line graph, also by month
#do a linear regression for cases vs.vaccines, deaths vs. vaccines, and possibly
#cases vs. deaths(though this one is very obvious)

#If there is time I can pull data from LA, Sacramento, and Humbolt counties and compare
#their vaccine rate (by population) to death and case rate

In [19]:
#import modules
import pandas as pd
import json
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import os
import csv
from pandas.api.types import CategoricalDtype
low_memory = False


In [20]:
#read in all of the csv files and convert them to data frames

#.csv is in the file name, so put .csv twice to read it
vac_county_path = './Resources/CHHS_Vaccines_Administered_by_County/130d7ba2-b6eb-438d-a412-741bde207e1c (1).csv.csv'
vac_by_county = pd.read_csv(vac_county_path)
vac_by_county = pd.DataFrame(vac_by_county)

vac_cleaned_path = './Resources/ca_fully_partially_vaccinated_by_month.csv'
vac_cleaned = pd.read_csv(vac_cleaned_path)
vac_cleaned = pd.DataFrame(vac_cleaned)

cases_county_path = './Resources/ca_cases_by_county/total_monthly_cases.csv'
cases_by_county = pd.read_csv(cases_county_path)
cases_by_county = pd.DataFrame(cases_by_county)

deaths_county_path = './Resources/CA_covid_deaths_by_county/ca_covid_deaths_by_county.csv'
deaths_by_county = pd.read_csv(deaths_county_path)
deaths_by_county = pd.DataFrame(deaths_by_county)

In [21]:
# State wide Covid-19 vaccines administered by county 
#this following section is basically what is in the beginning of Tikaram's notebook
#since he cleaned/worked with the vaccine data. If I have time for sample counties I will use this
#for now I'll just use the csv I wrote out from his notebook

# Converting the string format of date into datetime and created a new column for month
#vac_by_county['month'] = pd.DatetimeIndex(vac_by_county['administered_date']).month_name()

#vac_by_county = pd.DataFrame(vac_by_county, columns=['county','month','administered_date',
#                                                      'fully_vaccinated','partially_vaccinated'])

# Creating a new dataframe with months as variable to be utilized for ploting by months
#vac_by_county = pd.DataFrame(vac_by_county, columns=['county','month','fully_vaccinated','partially_vaccinated'])
#vac_by_county

In [22]:
print(cases_by_county)

     Months    Cases
0  December  1216142
1   January   873586
2  February   181288
3     March    82660
4     April    65037
5       May    37661
6      June    34899
7      July    71623


In [23]:

print(deaths_by_county)


                         County Name  August  September  October  November  \
0              Statewide Unallocated       0          0        0         0   
1                    Alameda County      257        412      463       512   
2                     Alpine County        0          0        0         0   
3                     Amador County       14         16       16        16   
4                      Butte County       23         44       53        60   
5                  Calaveras County        2         14       21        22   
6                     Colusa County        6          6        6         6   
7               Contra Costa County      175        209      246       261   
8                  Del Norte County        1          1        1         1   
9                  El Dorado County        2          4        4         4   
10                    Fresno County      263        390      443       481   
11                     Glenn County        3          3        3

In [ ]:
#create a dataframe for deaths by month for the entire state
#from December 2020 on so that it can be compared with vaccination
dec = 

In [16]:
print(vac_cleaned[200:])
vac_cleaned = vac_cleaned.dropna(how='any')
vac_cleaned

    month  fully_vaccinated  partially_vaccinated
200  July             25107                 21011
201  July              1887                  1515
202  July             21741                 18894
203  July             53036                 33129
204  July             42784                 30406
205  July             40780                 29376
206  July             46737                 35248
207  July             28801                 22467
208  July             12366                 10882
209  July             37950                 27634
210  July             37970                 31290
211  July             36704                 29712
212  July             35688                 31344
213  July             42281                 39475
214  July             27187                 25717
215  July             12614                 12758
216  July             33623                 32985
217  July             36679                 38456
218  July             35777                 37585


,month,fully_vaccinated,partially_vaccinated
0,December,0,1190
1,December,0,8767
2,December,0,26828
3,December,1,45622
4,December,1,25677
...,...,...,...
224,July,33675,46743
225,July,32184,47309
226,July,30277,46543
227,July,36620,54372
